In [6]:
import numpy as np
import pandas as pd
import sqlite3

%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
con = sqlite3.connect("test.sqlite")

In [8]:
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('preferences',), ('options',), ('picks',), ('cards',)]


### Pick Options

In [9]:
options = pd.read_sql_query("select * from options;", con)
options = options.set_index(['draft_id', 'drafter', 'pick_number'])

In [5]:
options

Angel of the Dawn  \
draft_id                             drafter pick_number                      
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                            2   
                                             1                            1   
                                             2                            2   
                                             3                            0   
                                             4                            1   
                                             5                            0   
                                             6                            0   
                                             7                            0   
                                             8                            0   
                                             9                            0   
                                             10                           0   
                                             11                           0   
                                             12                           0   
                                             13                           0   
                                             14                           1   
                                             15                           1   
                                             16                           1   
                                             17                           0   
                                             18                           1   
                                             19                           0   
                                             20                           0   
                                             21                           0   
                                             22                           0   
                                             23                           0   
                                             24                           0   
                                             25                           0   
                                             26                           0   
                                             27                           0   
                                             28                           1   
                                             29                           1   
...                                                                     ...   
                                     7       12                           0   
                                             13                           0   
                                             14                           1   
                                             15                           1   
                                             16                           0   
                                             17                           1   
                                             18                           0   
                                             19                           0   
                                             20                           0   
                                             21                           0   
                                             22                           0   
                                             23                           0   
                                             24                           0   
                                             25                           0   
                                             26                           0   
                                             27                           0   
                                             28                           1   
                                             29                           0   
                             

There should be a decreasing number of options as we progress through the draft, repeating in a cycle of length 14 (since there are 14 cards in each pack.

In [35]:
(options.sum(axis=1) == 14 - options.index.get_level_values('pick_number').values % 14).all()

True

To avoid duplicate column names, we will sanitize the names and add a prefix.

In [49]:
def sanitizer(prefix=''):
    def sanitize_name(name):
        return prefix + '_' + name.lower().replace(' ', '_')
    return sanitize_name

In [50]:
sanitizer(prefix='options')('Angel Of The Dawn')

'options_angel_of_the_dawn'

In [52]:
options.columns = options.columns.map(sanitizer(prefix='options'))

### Current Cards

In [55]:
cards = pd.read_sql_query("select * from cards;", con)
cards = cards.set_index(['draft_id', 'drafter', 'pick_number'])

In [56]:
cards

Angel of the Dawn  \
draft_id                             drafter pick_number                      
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                            0   
                                             1                            0   
                                             2                            0   
                                             3                            0   
                                             4                            0   
                                             5                            0   
                                             6                            0   
                                             7                            0   
                                             8                            0   
                                             9                            0   
                                             10                           0   
                                             11                           0   
                                             12                           0   
                                             13                           0   
                                             14                           0   
                                             15                           0   
                                             16                           0   
                                             17                           0   
                                             18                           0   
                                             19                           0   
                                             20                           0   
                                             21                           0   
                                             22                           0   
                                             23                           0   
                                             24                           0   
                                             25                           0   
                                             26                           0   
                                             27                           0   
                                             28                           0   
                                             29                           0   
...                                                                     ...   
                                     7       12                           0   
                                             13                           0   
                                             14                           0   
                                             15                           0   
                                             16                           0   
                                             17                           0   
                                             18                           0   
                                             19                           0   
                                             20                           0   
                                             21                           0   
                                             22                           0   
                                             23                           0   
                                             24                           0   
                                             25                           0   
                                             26                           0   
                                             27                           0   
                                             28                           0   
                                             29                           0   
                             

We should have an increasing number of cards as we progress through the draft. The number of cards held by each player should be equal to the pick number of the draft.

In [57]:
(cards.sum(axis=1) == cards.index.get_level_values('pick_number').values).all()

True

In [58]:
cards.columns = cards.columns.map(sanitizer(prefix='cards'))

### Draft Picks

Load the picks table, and convert it from a one-hot-encoded array into a series that tracks the pick by index (i.e., each card is assigned an integer).

In [60]:
picks = pd.read_sql_query("select * from picks;", con)
picks = picks.set_index(['draft_id', 'drafter', 'pick_number'])

In [61]:
pick_idx = pd.Series(np.argmax(picks.values, axis=1), index=picks.index)

In [62]:
pick_idx

draft_id                              drafter  pick_number
d82a6ded-407e-4ba7-be27-945ec7b9233e  0        0              19
                                               1              18
                                               2              18
                                               3              15
                                               4              15
                                               5              15
                                               6              16
                                               7              18
                                               8              16
                                               9              16
                                               10             16
                                               11             17
                                               12             18
                                               13             21
                               

### Joining

In [67]:
joined = pd.merge(cards, options, left_index=True, right_index=True)

In [68]:
joined["pick"] = pick_idx

In [69]:
joined.head()

cards_angel_of_the_dawn  \
draft_id                             drafter pick_number                            
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                  0   
                                             1                                  0   
                                             2                                  0   
                                             3                                  0   
                                             4                                  0   

                                                          cards_luminous_bonds  \
draft_id                             drafter pick_number                         
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                               0   
                                             1                               0   
                                             2                               0   
                                             3                               0   
                                             4                               0   

                                                          cards_pegasus_courser  \
draft_id                             drafter pick_number                          
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                0   
                                             1                                0   
                                             2                                0   
                                             3                                0   
                                             4                                0   

                                                          cards_hieromancer's_cage  \
draft_id                             drafter pick_number                             
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                   0   
                                             1                                   0   
                                             2                                   0   
                                             3                                   0   
                                             4                                   0   

                                                          cards_leonin_warleader  \
draft_id                             drafter pick_number                           
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                 0   
                                             1                                 0   
                                             2                                 0   
                                             3                                 0   
                                             4                                 0   

                                                          cards_aviation_pioneer  \
draft_id                             drafter pick_number                           
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                 0   
                                             1                                 0   
                                             2                                 0   
                                             3                                 0   
                                             4                                 0   

                                                          cards_essence_scatter  \
draft_id                             drafter pick_number                          
d82a6ded-407e-4ba7-be27-945ec7b9233e 0       0                                0   
                                             1                                0   
                                             2                                0   
                                             3                                0   
                          